<h2>Usage of self.model attribute</h2>
<p>Using callback functions, we received log information by calling methods at taining/testing of the model. In adiition to this, callbacks have access to the model associated with the current round of training/evaluation/inference by using <b>self.model</b>. Here are the few things we can do using self.model in callback:</p>
<ul><li>We can Set <b>self.model.stop_training = True</b> to immediately interrupt training.</li>
<li>We can mutate hyperparameters of the optimizer (available as self.model.optimizer) such as <b>self.model.optimizer.learning_rate</b>.</li>
<li>Save the model at period intervals.</li>
<li>We can record the output of <b>model.predict()</b> on a few test samples at the end of each epoch, to use as a sanity check during training.</li>
<li>We can extract visualizations of intermediate features at the end of each epoch, to monitor what the model is learning over time.</li>
<li>and so on.</li></ul>
<p>We will see these things using an example.</p>

<h2>Examples of Keras callback applications</h2>
<h4>Early stopping at minimum loss</h4>
<p>In this example, we show the creation of callback that stops training when the loss reached minimum value, by setting the attribute <b>self.model.stop_training(boolean)</b>. Optionally, we can provide an arguement <b>patience</b> to specify after reached a local minimum, how many epochs we should wait before stopping. </p>

In [1]:
# Importing required libraries
import tensorflow as tf
from tensorflow import keras

# Definingthe function for creating simple sequential model, compiling it and return the model 
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    # Compiling the model with RMSprop optimizer of keras
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

# Load example MNIST data 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#pre-process the data
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

11493376/11490434 [==============================] - 0s 0us/step


In [2]:
#Defining custom callback LossAndErrorPrinting Callback(which is defined in previous notebook)
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
  # defining callback method for end of the training batch
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

# defining callback method for end of the testing batch
    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

# defining callback method for end of an epoch
    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )

In [3]:
import numpy as np

# Creating EarlyStoppingAtMinLoss class which extends keras.callback.Callback
class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))

# Creating the model
model = get_model()

# Training the model with batch size 64 and 2 callbacks.
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossAndErrorPrintingCallback(), EarlyStoppingAtMinLoss()],
)

For batch 0, loss is   26.40.
For batch 1, loss is  476.68.
For batch 2, loss is  324.73.
For batch 3, loss is  245.30.
For batch 4, loss is  197.95.
The average loss for epoch 0 is  197.95 and mean absolute error is    8.24.
For batch 0, loss is    8.20.
For batch 1, loss is    7.11.
For batch 2, loss is    6.86.
For batch 3, loss is    6.43.
For batch 4, loss is    6.42.
The average loss for epoch 1 is    6.42 and mean absolute error is    2.06.
For batch 0, loss is    4.02.
For batch 1, loss is    4.12.
For batch 2, loss is    4.72.
For batch 3, loss is    4.94.
For batch 4, loss is    5.46.
The average loss for epoch 2 is    5.46 and mean absolute error is    1.92.
For batch 0, loss is    6.74.
For batch 1, loss is    7.82.
For batch 2, loss is    9.79.
For batch 3, loss is   10.15.
For batch 4, loss is   11.95.
The average loss for epoch 3 is   11.95 and mean absolute error is    2.84.
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


<h4>Learning rate scheduling</h4>
<p>In the second example, we show how a custom callback can be used to change the learning rate of the optimizer dynamically during training.</p> 

In [4]:
# Defining the callback class CustomLearningRateScheduler
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

# Callback method for epoch begin
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


# Defining the tuple for lr schedule
LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (3, 0.05),
    (6, 0.01),
    (9, 0.005),
    (12, 0.001),
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

# Create the model
model = get_model()

# Training the model with batch size 64 and 15 epochs
model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=15,
    verbose=0,
    callbacks=[
        LossAndErrorPrintingCallback(),
        CustomLearningRateScheduler(lr_schedule),
    ],
)


Epoch 00000: Learning rate is 0.1000.
For batch 0, loss is   25.94.
For batch 1, loss is  433.42.
For batch 2, loss is  297.16.
For batch 3, loss is  225.38.
For batch 4, loss is  181.89.
The average loss for epoch 0 is  181.89 and mean absolute error is    8.16.

Epoch 00001: Learning rate is 0.1000.
For batch 0, loss is    6.08.
For batch 1, loss is    6.40.
For batch 2, loss is    6.21.
For batch 3, loss is    5.98.
For batch 4, loss is    5.82.
The average loss for epoch 1 is    5.82 and mean absolute error is    1.93.

Epoch 00002: Learning rate is 0.1000.
For batch 0, loss is    4.71.
For batch 1, loss is    5.04.
For batch 2, loss is    4.38.
For batch 3, loss is    4.03.
For batch 4, loss is    4.14.
The average loss for epoch 2 is    4.14 and mean absolute error is    1.65.

Epoch 00003: Learning rate is 0.0500.
For batch 0, loss is    5.44.
For batch 1, loss is    4.18.
For batch 2, loss is    4.78.
For batch 3, loss is    4.62.
For batch 4, loss is    4.67.
The average loss